<a href="https://colab.research.google.com/github/JerryChenz/InvestmentManagement/blob/master/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Set inputs

We load the sample dataset from the my github repository, and display the dataset in pandas.

In [ ]:
import pandas as pd
summary_url = 'https://raw.githubusercontent.com/JerryChenz/InvestmentManagement/master/financial_models/Opportunities/Monitor/screener_summary.csv'
df = pd.read_csv(summary_url)
display(df)

In [ ]:
# capitalization in reporting currency
capitalization = df['Price'] * df['Shares'] * df['Fx_rate']
total_debt = df['CurrentDebtAndCapitalLeaseObligation'] + df['LongTermDebtAndCapitalLeaseObligation']
enterprise_value = capitalization + total_debt + df['MinorityInterest'] - df['CashAndCashEquivalents']
# more easily realizable non-operating assets
monetary_assets = df['OtherShortTermInvestments'] + df['InvestmentinFinancialAssets']
# less easily realizable non-operating assets
fixed_nonop_assets = (df['InvestmentProperties'] + df['LongTermEquityInvestment']) * 0.5
# liquidity_coverage_ratio
lcr = monetary_assets / df['CurrentLiabilities']
current_ratio = df['CurrentAssets'] / df['CurrentLiabilities']

,Ticker,TotalAssets,CurrentAssets,CurrentLiabilities,CurrentDebtAndCapitalLeaseObligation,CurrentCapitalLeaseObligation,LongTermDebtAndCapitalLeaseObligation,LongTermCapitalLeaseObligation,TotalEquityGrossMinorityInterest,MinorityInterest,...,NetIncomeCommonStockholders,Name,Exchange,Price,Price_currency,Shares,Reporting_Currency,Fx_rate,Last_fy,Dividend
0,0806.HK,5.670342e+09,1.971613e+09,2.859090e+08,19771000.0,19771000.0,108512000.0,25878000.0,5.264755e+09,0.0,...,457813000.0,VALUE PARTNERS,HKG,3.05,HKD,1826710016,HKD,1,2021-12-31,0.345281
1,1475.HK,5.259569e+09,3.108899e+09,1.060434e+09,5576000.0,5576000.0,5636000.0,5636000.0,4.116261e+09,165622000.0,...,303821000.0,NISSIN FOODS,HKG,6.72,HKD,1043689984,HKD,1,2021-12-31,0.143857


# Step 2: Screening Criteria

In [ ]:
# filter conditions
condition_1 = df['CashAndCashEquivalents'] > df['CurrentDebtAndCapitalLeaseObligation']
condition_2 = lcr >= 1 

#Step 3. Output

Filter the dataset using the above conditions

In [ ]:
# Result df
df.loc[condition_1 &condition_2]

,Ticker,TotalAssets,CurrentAssets,CurrentLiabilities,CurrentDebtAndCapitalLeaseObligation,CurrentCapitalLeaseObligation,LongTermDebtAndCapitalLeaseObligation,LongTermCapitalLeaseObligation,TotalEquityGrossMinorityInterest,MinorityInterest,...,NetIncomeCommonStockholders,Name,Exchange,Price,Price_currency,Shares,Reporting_Currency,Fx_rate,Last_fy,Dividend
0,0806.HK,5.670342e+09,1.971613e+09,285909000.0,19771000.0,19771000.0,108512000.0,25878000.0,5.264755e+09,0.0,...,457813000.0,VALUE PARTNERS,HKG,3.05,HKD,1826710016,HKD,1,2021-12-31,0.345281
